In [96]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [97]:
data = pd.read_csv('data/train.csv')

In [98]:
data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [99]:
data = data[data['MISSING_DATA'] == False]

In [100]:
def polyline_to_trip_duration(polyline):
    return max(polyline.count("[") - 2, 0) * 15 #subtracting 2 because one is for the opening bracket?

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
data["LEN"] = data["POLYLINE"].apply(polyline_to_trip_duration)

In [101]:
def parse_time(x):
    dt = datetime.fromtimestamp(x["TIMESTAMP"])
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

In [102]:
data[["YR", "MON", "DAY", "HR", "WK"]] = (
    data[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
)

In [103]:
#filling nan's with -1 to avoid errors
data['ORIGIN_STAND'].fillna(-1, inplace = True)

In [104]:
type_map = {'A': 1, 'B': 2, 'C': 3}
data['CALL_TYPE'] = data['CALL_TYPE'].map(type_map).astype(int) 
data['DAY_TYPE'] = data["DAY_TYPE"].map(type_map).astype(int) 

In [105]:
test_data = pd.read_csv('data/test_public.csv')

In [106]:
test_data

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,T1,B,NaN,15.0,20000542,1408039037,A,False
1,T2,B,NaN,57.0,20000108,1408038611,A,False
2,T3,B,NaN,15.0,20000370,1408038568,A,False
3,T4,B,NaN,53.0,20000492,1408039090,A,False
4,T5,B,NaN,18.0,20000621,1408039177,A,False
...,...,...,...,...,...,...,...,...
315,T323,A,70885.0,NaN,20000430,1419171485,A,False
316,T324,B,NaN,53.0,20000020,1419170802,A,False
317,T325,C,NaN,NaN,20000207,1419172121,A,False
318,T326,A,76232.0,NaN,20000667,1419171980,A,False


In [107]:
test_data[["YR", "MON", "DAY", "HR", "WK"]] = (
    test_data[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
)

In [108]:
print(test_data.to_string())

    TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID   TIMESTAMP DAY_TYPE  MISSING_DATA    YR  MON  DAY  HR  WK
0        T1         B          NaN          15.0  20000542  1408039037        A         False  2014    8   14  10   3
1        T2         B          NaN          57.0  20000108  1408038611        A         False  2014    8   14  10   3
2        T3         B          NaN          15.0  20000370  1408038568        A         False  2014    8   14  10   3
3        T4         B          NaN          53.0  20000492  1408039090        A         False  2014    8   14  10   3
4        T5         B          NaN          18.0  20000621  1408039177        A         False  2014    8   14  10   3
5        T6         A      42612.0           NaN  20000607  1408037146        A         False  2014    8   14  10   3
6        T7         B          NaN          15.0  20000310  1408038846        A         False  2014    8   14  10   3
7        T8         A      31780.0           NaN  200006

In [109]:
data.shape

(1710660, 15)

In [111]:
filtered = data[data["YR"] == 2014]

In [112]:
filtered.shape

(847980, 15)

In [ ]:

# Assuming you have a DataFrame called 'df' with the columns ["YR", "MON", "DAY", "HR", "WK"]

# Define a list of holiday dates
holidays = [
    {"MON": 1, "DAY": 1},  # New Year's Day
    {"MON": 1, "DAY": 20},  # Independence Day
    {"MON": 2, "DAY": 17},
    {"MON": 5, "DAY": 26},
    {"MON": 7, "DAY": 4},
    {"MON": 9, "DAY": 1},
    {"MON": 10, "DAY": 13},
    {"MON": 11, "DAY": 11},
    {"MON": 11, "DAY": 27},
    {"MON": 12, "DAY": 25},
]


In [89]:
#Dataset starts month 6/2013
#monhts missing from 2014: 7 -> 12
#test set months: 8, 9, 10, 12
data.loc[(data['MON']== 12) & (data['YR'] == 2014)]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LEN,YR,MON,DAY,HR,WK


In [ ]:
data.loc[((data['MON']== 7) | (data['MON']== 8) | (data['MON']== 9) | (data['MON']== 10) | (data['MON']== 11) | (data['MON']== 12)) & (data['YR'] == 2013)]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LEN,YR,MON,DAY,HR,WK
443,1372662532620000051,2,NaN,34.0,20000051,1372662532,1,False,"[[-8.615601,41.140692],[-8.615349,41.140854],[...",585,2013,7,1,0,0
450,1372662153620000351,3,NaN,-1.0,20000351,1372662153,1,False,"[[-8.670015,41.236965],[-8.669727,41.235633],[...",915,2013,7,1,0,0
454,1372663182620000247,1,61064.0,-1.0,20000247,1372663182,1,False,"[[-8.593767,41.157639],[-8.594208,41.156982],[...",585,2013,7,1,0,0
464,1372662071620000662,3,NaN,-1.0,20000662,1372662071,1,False,"[[-8.641611,41.156559],[-8.641431,41.156541],[...",315,2013,7,1,0,0
465,1372662631620000621,1,46490.0,-1.0,20000621,1372662631,1,False,"[[-8.611902,41.159079],[-8.611911,41.159061],[...",570,2013,7,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710624,1373653545620000940,1,2002.0,-1.0,20000940,1373653545,1,False,"[[-8.628561,41.158998]]",0,2013,7,12,11,4
1710643,1386603894620000970,3,NaN,-1.0,20000970,1386603894,1,False,[],0,2013,12,9,7,0
1710657,1380123541620000980,3,NaN,-1.0,20000980,1380123541,1,False,"[[-8.689959,41.240241]]",0,2013,9,25,8,2
1710658,1373986578620000931,1,2002.0,-1.0,20000931,1373986578,1,False,[],0,2013,7,16,7,1


In [115]:
data.loc[((data['MON']== 12)) & (data['YR'] == 2013)]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LEN,YR,MON,DAY,HR,WK
714397,1385885185620000367,2,NaN,57.0,20000367,1385885185,1,False,"[[-8.61093,41.145786],[-8.610921,41.145795],[-...",300,2013,12,1,0,6
714398,1385885042620000305,2,NaN,57.0,20000305,1385885042,1,False,"[[-8.610759,41.145687],[-8.610759,41.14593],[-...",360,2013,12,1,0,6
714403,1385885225620000545,2,NaN,57.0,20000545,1385885225,1,False,"[[-8.610705,41.145561],[-8.610732,41.145552],[...",375,2013,12,1,0,6
714409,1385884964620000263,3,NaN,-1.0,20000263,1385884964,1,False,"[[-8.613567,41.146947],[-8.613558,41.146776],[...",540,2013,12,1,0,6
714411,1385885168620000148,2,NaN,36.0,20000148,1385885168,1,False,"[[-8.649486,41.154183],[-8.649612,41.15421],[-...",405,2013,12,1,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957860,1388419156620000969,3,NaN,-1.0,20000969,1388419156,1,False,"[[-8.64486,41.159088]]",0,2013,12,30,7,0
978668,1387585767620000344,2,NaN,10.0,20000344,1387585767,1,False,"[[-8.606952,41.150196],[-8.607006,41.150205],[...",1155,2013,12,20,16,4
1012755,1387213521620000521,2,NaN,17.0,20000521,1387213521,1,False,"[[-8.632287,41.164443],[-8.632746,41.164083],[...",960,2013,12,16,9,0
1179938,1387437315620000185,3,NaN,-1.0,20000185,1387437315,1,False,"[[-8.585622,41.127597],[-8.58456,41.127984],[-...",1560,2013,12,18,23,2
